I am losing precision from loading directly from the .feather file. 
Thus instead of loading diretly from there, the idea is to instantiate a sweep file and then get its data. 

The only problem is that I don't know if the points in a sweep and its laser_ids and intensity values have 1-1 correspondence. So I will test this out using this notebook

In [1]:
import os
from av2.datasets.sensor.av2_sensor_dataloader import AV2SensorDataLoader
from pathlib import Path
home = os.path.expanduser("~")

dataset_path = os.path.join(home, "buni", "dataset", "av2-pixor", "overfit", "train")
dataset_path = Path(dataset_path)
dataloader = AV2SensorDataLoader(data_dir=dataset_path, labels_dir=dataset_path)

log_id = 'ff6adc87-5f47-32f7-b36a-546453c0e332'

print(dataloader.get_log_ids())
print(dataloader.get_ordered_log_lidar_timestamps(log_id))

['ff6adc87-5f47-32f7-b36a-546453c0e332']
[315968449159644000]


In [2]:
timestamp = 315968449159644000
frame_path = dataloader.get_lidar_fpath(log_id,timestamp)

import pandas as pd

df = pd.read_feather(frame_path)

In [3]:
from av2.structures.sweep import Sweep

sweep = Sweep.from_feather(frame_path)

points = sweep.xyz
intensities = sweep.intensity
laser_numbers = sweep.laser_number

print(f"points shape: {points.shape} | intensities shape: {intensities.shape} | laser_numbers shape: {laser_numbers.shape}")

points shape: (96884, 3) | intensities shape: (96884,) | laser_numbers shape: (96884,)


In [4]:
import numpy as np
frame_final = np.column_stack((points, intensities, laser_numbers))

print(f"frame_final shape: {frame_final.shape}")

frame_final shape: (96884, 5)


In [5]:
import numpy as np

# Example arrays
array1 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
array2 = np.array([10, 11, 12])
array3 = np.array([13, 14, 15])

# Adding array2 and array3 as columns to array1
result = np.column_stack((array1, array2, array3))

print(result)

[[ 1  2  3 10 13]
 [ 4  5  6 11 14]
 [ 7  8  9 12 15]]


# Conclusion

With this I conclude that it is better to instantiate sweep and then create a numpy array than using a pandas dataframe